In [1]:
from rdkit.Chem import Draw
from rdkit import Chem

In [2]:
import os
os.chdir('../programming_scripts/.')
print(os.getcwd())

from MS2LDA_core import run_lda
from MS2LDA_core import predict_with_lda

#from Spec2Vec_annotation import motif_annotation

from MS2LDA_core import load_mgf
from MS2LDA_core import clean_spectra

from utils import motifs2spectra
from spec2vec import Spec2Vec

C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\Project_SubstructureIdentification\scripts\programming_scripts


In [5]:
# spectra_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\Project_SubformulaAnnotation\raw_data\_RAWdata1\GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf"
# spectra_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\Project_SubformulaAnnotation\raw_data\_RAWdata1\GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf"
# spectra_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\Project_SubformulaAnnotation\raw_data\_RAWdata1\GNPS-SCIEX-LIBRARY.mgf"
spectra_path = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\MEDUSA\notebooks\MS2LDA\PDE5_standards_annotated_pos_unique.mgf"

In [6]:
lda_model, corpus, id2word = run_lda(spectra_path=spectra_path, num_motifs=65, iterations=300)

In [7]:
motif_spectra = motifs2spectra(lda_model)

### how to load the embeddings and then still retrieve the smiles afterwards

In [8]:
import pandas as pd

In [9]:
from Spec2Vec_annotation import load_model_and_data
from Spec2Vec_annotation import calc_similarity
from Spec2Vec_annotation import retrieve_top_hits

In [10]:
s2v_model, embeddings_smiles_DB = load_model_and_data()

In [11]:
smiles = embeddings_smiles_DB.smiles.to_numpy()
embeddings_DB = embeddings_smiles_DB.embeddings.to_list()
spectra = embeddings_smiles_DB.spectra.to_list()

In [12]:
s2v_scores = calc_similarity(s2v_model, motif_spectra, embeddings_DB)

AssertionError: Weights are not normalized to unity as expected.

In [ ]:
top_scores, top_smiles = retrieve_top_hits(s2v_scores, 5, smiles)

In [ ]:
top_scores

In [ ]:
top_smiles

---

### New Part: Retrieve overlaping fragments between spectra with motif spectra

In [14]:
from utils import match_frags_and_losses

In [15]:
# just to get some spectra: it is really important that the spectra are binned
smiles_per_motifs, predicted_motifs, predicted_motifs_distribution, spectra_per_motifs = predict_with_lda(lda_model, spectra_path, id2word)

In [16]:
i = 5

In [17]:
matching_frags, matching_losses = match_frags_and_losses(motif_spectra[i], spectra_per_motifs[i])

In [18]:
matching_frags

[{58.06, 99.09, 491.19},
 {58.06, 99.09, 154.67, 169.1, 505.23},
 {58.06, 99.09, 154.67, 169.1, 489.23},
 {58.06, 99.09, 154.67, 532.26},
 {135.04, 154.67, 204.08, 262.09}]

In [19]:
matching_losses

[set(), set(), set(), set(), set()]

### New Part: Use MassQL to search for matches

In [20]:
from MassQL_annotation import initialize_massql_DB
from MassQL_annotation import add_column
from MassQL_annotation import search_massql_DB

In [24]:
spectra_path_GNPS = r"C:\Users\dietr004\Documents\PhD\computational mass spectrometry\Spec2Struc\Project_SubstructureIdentification\raw_data\GNPS\cleaned_spectra_min_5_peaks.mgf"

In [25]:
ms1_df, ms2_df = initialize_massql_DB(spectra_path_GNPS)

In [35]:
ms1_df

,i,i_norm,i_tic_norm,mz,scan,rt,precmz,ms1scan,charge,polarity
0,0.008392,0.008392,0.006534,173.521896,7232,0,0,0,1,1


In [43]:
ms2_df

,i,i_norm,i_tic_norm,mz,scan,rt,precmz,ms1scan,charge,polarity
0,0.010553,0.010553,0.000843,289.286377,1,0,0,0,1,1
1,0.029439,0.029439,0.002352,295.545288,1,0,0,0,1,1
2,0.005134,0.005134,0.000410,298.489624,1,0,0,0,1,1
3,0.024515,0.024515,0.001959,317.324951,1,0,0,0,1,1
4,0.011254,0.011254,0.000899,319.655945,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
21288060,1.000000,1.000000,0.778560,74.024498,7232,0,0,0,1,1
21288061,0.163899,0.163899,0.127606,74.029800,7232,0,0,0,1,1
21288062,0.040293,0.040293,0.031371,74.032402,7232,0,0,0,1,1
21288063,0.029471,0.029471,0.022945,128.107895,7232,0,0,0,1,1


In [59]:
ms2_df.loc[ms2_df["scan"] == 1]

,i,i_norm,i_tic_norm,mz,scan,rt,precmz,ms1scan,charge,polarity
0,0.010553,0.010553,0.000843,289.286377,1,0,0,0,1,1
1,0.029439,0.029439,0.002352,295.545288,1,0,0,0,1,1
2,0.005134,0.005134,0.000410,298.489624,1,0,0,0,1,1
3,0.024515,0.024515,0.001959,317.324951,1,0,0,0,1,1
4,0.011254,0.011254,0.000899,319.655945,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
21163387,0.068055,0.068055,0.025530,345.151093,1,0,0,0,1,1
21163388,1.000000,1.000000,0.375147,361.182404,1,0,0,0,1,1
21163389,0.067293,0.067293,0.025245,362.185913,1,0,0,0,1,1
21163390,0.008383,0.008383,0.003145,391.191010,1,0,0,0,1,1


In [26]:
# store ms1_df and ms2_df right away!!!

In [30]:
ms1_df.to_parquet("ms1_massql.parquet")

In [31]:
ms2_df.to_parquet("ms2_massql.parquet")

In [32]:
def generate_query(i):
    input_query_base = "QUERY scaninfo(MS2DATA) WHERE"
    input_query = "QUERY scaninfo(MS2DATA) WHERE"
    for matching_frag in matching_frags[i]:
        if input_query == input_query_base:
            input_query += f" MS2PROD={matching_frag}"
        else:
            input_query += f" AND MS2PROD={matching_frag}"
    for matching_loss in matching_losses[i]:
         if input_query == input_query_base:
            input_query += f" MS2PROD={matching_loss}"
         else:
            input_query += f" AND MS2PROD={matching_loss}"

    return input_query

In [55]:
input_query = generate_query(1)

In [56]:
input_query

'QUERY scaninfo(MS2DATA) WHERE MS2PROD=99.09 AND MS2PROD=58.06 AND MS2PROD=169.1 AND MS2PROD=505.23 AND MS2PROD=154.67'

In [39]:
from massql import msql_engine

In [57]:
results_df = msql_engine.process_query(input_query, spectra_path_GNPS, ms1_df=ms1_df, ms2_df=ms2_df)

TOTAL QUERIES 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.83s/it]


In [58]:
results_df

,scan,precmz,ms1scan,rt,charge,i,i_norm,mslevel,i_norm_ms1
0,1,0,0,0,1,20370.455346,1.0,2,NaN
1,2,0,0,0,1,735.131476,1.0,2,NaN
2,3,0,0,0,1,1252.619839,1.0,2,NaN
3,6,0,0,0,1,485.094371,1.0,2,NaN
4,8,0,0,0,1,300.356656,1.0,2,NaN
...,...,...,...,...,...,...,...,...,...
357,7895,0,0,0,1,15.128028,1.0,2,NaN
358,8168,0,0,0,1,31.133472,1.0,2,NaN
359,9537,0,0,0,1,26.525331,1.0,2,NaN
360,9925,0,0,0,1,25.420851,1.0,2,NaN


In [36]:
# single test:
input_query = generate_query(1)
results_df = search_massql_DB(spectra_path_GNPS, input_query, ms1_df, ms2_df)
print(results_df.to_numpy())

TOTAL QUERIES 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.79s/it]


KeyboardInterrupt: 

In [23]:
for i in range(len(matching_frags)):
    input_query = generate_query(i)
    results_df = search_massql_DB(spectra_path, input_query, ms1_df, ms2_df)
    print(results_df.to_numpy())

TOTAL QUERIES 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 40.76it/s]


[[594 491.18946 0 674.1 1 3.3262899999999997 1.0 2 nan
  'CCCC1=NN(C2=C1NC(=NC2=S)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)C)OCC)C']]
TOTAL QUERIES 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 67.33it/s]


[[215 505.22332 0 539.78 1 2.53269 1.0 2 nan
  'CCCC1=NN(C2=C1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)CCO)OCC)C']]
TOTAL QUERIES 1


...

[[222 489.22767 0 540.68 1 2.4637499999999997 1.0 2 nan
  'CCCC1=NC(=C2N1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)CC)OCC)C']]
TOTAL QUERIES 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 81.63it/s]

[[483 532.25912 0 647.78 1 4.54353 1.0 2 nan
  'CCCC1=CN(C2=C1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)CCO)OCCC)CC']]
TOTAL QUERIES 1



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 99.99it/s]

[[313 390.14504 0 622.64 1 4.5408100000000005 1.0 2 nan
  'CN1CC(=O)N2[C@H](Cc3c([nH]c4ccccc34)[C@H]2c2ccc3c(c2)OCO3)C1=O']
 [321 391.14041 0 584.59 1 3.836 1.0 2 nan
  'C1C2C(=O)N(CC(=O)N2C(C3=C1C4=CC=CC=C4N3)C5=CC6=C(C=C5)OCO6)N']
 [420 433.15077 0 588.35 1 4.25434 1.0 2 nan
  'CC(=O)NN1CC(=O)N2C(C1=O)CC3=C(C2C4=CC5=C(C=C4)OCO5)NC6=CC=CC=C36']]
